In [6]:
import xarray as xr
import numpy as np
import pandas as pd

'''
# Example: creating an xarray Dataset with some float data
timestamps = pd.date_range('2024-01-01', periods=500, freq='H')  # 500 timestamps
data = np.random.randn(500)  # 500 random float data values

ds = xr.Dataset({
    'data': ('time', data)
}, coords={'time': timestamps})

ds
'''

# Example: Create the xarray Dataset with 500 timestamps and 2 channels
timestamps = pd.date_range('2024-01-01', periods=500, freq='H')  # 500 timestamps
channels = ['channel_1', 'channel_2']  # 2 channels
data = np.random.randn(2,500)  # 500 timestamps, 2 channels (random data)

ds = xr.Dataset({
    'data': (('channel_id', 'time'), data)
}, coords={'channel_id':channels, 'time': timestamps})

ds

C:\Users\lukas\AppData\Local\Temp\ipykernel_2188\2733945423.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range('2024-01-01', periods=500, freq='H')  # 500 timestamps


<xarray.Dataset> Size: 12kB
Dimensions:     (channel_id: 2, time: 500)
Coordinates:
  * channel_id  (channel_id) <U9 72B 'channel_1' 'channel_2'
  * time        (time) datetime64[ns] 4kB 2024-01-01 ... 2024-01-21T19:00:00
Data variables:
    data        (channel_id, time) float64 8kB -0.1108 0.8176 ... -1.104 1.336

In [8]:
# Step 1: Reshape the data into chunks of 100
num_groups = len(ds.time) // 100  # number of groups (e.g., 500 samples -> 5 groups)

# Reshape the data to a new shape with the group dimension
data_reshaped = ds['data'].values.reshape(num_groups, 2, 100)

data_reshaped

array([[[-1.10788784e-01,  8.17582142e-01, -7.69666967e-01,
         -1.64466994e-01,  1.43391703e-01,  5.68112596e-01,
         -1.47340165e+00, -1.56889962e+00,  2.55762072e-01,
          6.41934680e-01, -2.37714270e+00,  1.78756307e+00,
         -4.95730517e-01, -1.10157671e+00,  1.69289426e+00,
          1.29810729e+00, -4.55224074e-01,  1.01900499e+00,
          1.15312723e-01,  2.36613957e+00, -2.85343334e-01,
         -1.12268321e+00,  1.82511984e+00, -8.17566169e-01,
          2.42232147e-01, -8.10584796e-02, -1.18804961e+00,
          1.13118569e+00, -4.19216933e-01,  2.71392512e+00,
          1.57656347e+00, -1.98691031e+00, -5.80754688e-01,
         -4.28154276e-01, -4.63623196e-01,  1.36870852e-01,
          1.37044985e+00,  9.18947572e-01,  7.05346055e-01,
         -2.65776114e-01, -1.23763232e+00, -2.16716482e-01,
         -4.74207228e-01, -2.64115843e-01, -1.61251565e+00,
         -1.96275587e-01, -1.70885575e+00,  6.95831391e-02,
         -2.60976832e-01,  1.16645608e+0

In [9]:
# Step 2: Create a new coordinate for group_id
group_id = np.arange(num_groups)
group_id

array([0, 1, 2, 3, 4])

In [11]:
# Step 4: Create the new Dataset with `group_id` as a dimension
ds_new = xr.Dataset({
    'data': (('group_id', 'channel', 'time_within_group'), data_reshaped)
}, coords={
    'group_id': group_id,
    'channel': channels,  # 2 channels
    'time_within_group': np.arange(100)  # 100 timestamps within each group 
})



# Print the reshaped dataset
ds_new

<xarray.Dataset> Size: 9kB
Dimensions:            (group_id: 5, channel: 2, time_within_group: 100)
Coordinates:
  * group_id           (group_id) int64 40B 0 1 2 3 4
  * channel            (channel) <U9 72B 'channel_1' 'channel_2'
  * time_within_group  (time_within_group) int64 800B 0 1 2 3 4 ... 96 97 98 99
Data variables:
    data               (group_id, channel, time_within_group) float64 8kB -0....

### extracting coordinate as a value

In [13]:
# Example: Create a simple Dataset with time and group_id as coordinates
timestamps = pd.date_range('2024-01-01', periods=5, freq='H')  # 5 timestamps
group_id = np.array([0, 0, 1, 1, 2])  # A small group_id array

ds = xr.Dataset({
    'data': ('time', np.random.randn(5))
}, coords={'time': timestamps, 'group_id': group_id})

# Extracting a single value from a coordinate
single_time_value = ds.time.values[0]  # Access the first timestamp
single_group_id_value = ds.group_id.values[3]  # Access the first group_id value

print("Single timestamp value:", single_time_value)
print("Single group_id value:", single_group_id_value)

Single timestamp value: 2024-01-01T00:00:00.000000000
Single group_id value: 1


C:\Users\lukas\AppData\Local\Temp\ipykernel_2188\965565336.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range('2024-01-01', periods=5, freq='H')  # 5 timestamps
